In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from tqdm import tqdm
import ast

import random

from multiprocessing import Pool

In [2]:
pares_positivos = pd.read_csv('../../Data/pares_positivos_ids.csv')
pares_positivos

,Id_1,Id_2
0,9655,15085
1,15654,18626
2,4026,2507
3,2507,18316
4,4026,24911
...,...,...
4513623,26067,16631
4513624,14303,16631
4513625,19539,16631
4513626,20409,16631


In [3]:
pares_negativos = pd.read_csv('../../Data/pares_negativos_ids.csv')
pares_negativos

,Id_1,Id_2
0,2379,9050
1,10138,9537
2,12381,1716
3,5126,7195
4,16240,23369
...,...,...
4513623,24662,9427
4513624,11572,18734
4513625,9809,20969
4513626,25278,27264


In [4]:
items_onehot_encoded_df = pd.read_csv('../../Data/items_onehot_encoded.csv')
items_onehot_encoded_df

,item_id,1_60,1_143,1_358,1_461,1_517,1_771,2_53,2_56,2_83,...,72_665,72_671,72_712,72_747,72_748,72_751,72_829,72_883,73_91,73_544
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23687,28140,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
23688,28141,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23689,28142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


def create_dataframe_for_training(pares_positivos_df, pares_negativos_df):
    
    pares_positivos_df['Y'] = 1
    pares_negativos_df['Y'] = 0

    data_df = pd.concat([pares_positivos_df,pares_negativos_df],ignore_index=True)
    data_df_shuffle = data_df.sample(frac=1).reset_index(drop=True)

    return data_df_shuffle

training_ids_item2vec_df = create_dataframe_for_training(pares_positivos, pares_negativos)
training_ids_item2vec_df

training_ids_item2vec_df.to_csv('../../Data/training_ids_item2vec_df.csv', index=False)

In [11]:
training_ids_item2vec_df = pd.read_csv('../../Data/training_ids_item2vec_df.csv')
training_ids_item2vec_df

,Id_1,Id_2,Y
0,856,2393,0
1,27082,19579,0
2,1652,25966,1
3,8426,13996,0
4,1473,13615,1
...,...,...,...
9027251,5465,10434,1
9027252,16097,23946,1
9027253,5199,3037,1
9027254,12604,10772,1


In [7]:
def return_numpy_onehot_from_item_id(item_id, one_hot_items):
    
    item_one_hot_df = one_hot_items.loc[one_hot_items.item_id == item_id].drop('item_id', axis=1)
    
    return item_one_hot_df.to_numpy()
    

def create_vectos_training_embedding(pares_df, one_hot_items):
    
    X1 = []
    X2 = []
    Y = []
    
    for index, row in tqdm(pares_df.iterrows()):
        
        #X1
        X1.append(return_numpy_onehot_from_item_id(row.Id_1, one_hot_items))
        
        #X2
        X2.append(return_numpy_onehot_from_item_id(row.Id_2, one_hot_items))
        
        #Y
        Y.append(row.Y)
        

    X1 = np.array(X1)
    X2 = np.array(X2)
    Y = np.array(Y)
    
    X1 = np.reshape(X1, (X1.shape[0], X1.shape[2]))
    X2 = np.reshape(X2, (X2.shape[0], X2.shape[2]))
    Y = np.reshape(Y, (Y.shape[0], 1))
    
    return X1, X2, Y

# Parte 1

In [8]:
training_ids_item2vec_df.iloc[:500000]

,Id_1,Id_2,Y
0,856,2393,0
1,27082,19579,0
2,1652,25966,1
3,8426,13996,0
4,1473,13615,1
...,...,...,...
499995,5097,1725,0
499996,20565,1799,1
499997,17780,11959,1
499998,20435,17940,0


In [9]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[:500000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_1', X1_onehot)
np.save('../../Data/X2_onehot_1', X2_onehot)
np.save('../../Data/Y_1', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:55, 1203.35it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 2

In [12]:
training_ids_item2vec_df.iloc[500000:1000000]

,Id_1,Id_2,Y
500000,19348,15531,0
500001,13048,868,1
500002,26982,15881,1
500003,19540,4534,1
500004,25355,2780,1
...,...,...,...
999995,6381,18313,0
999996,13613,24784,0
999997,21786,6355,1
999998,16050,7383,1


In [13]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[500000:1000000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_2', X1_onehot)
np.save('../../Data/X2_onehot_2', X2_onehot)
np.save('../../Data/Y_2', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:51, 1214.02it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 3

In [14]:
training_ids_item2vec_df.iloc[1000000:1500000]

,Id_1,Id_2,Y
1000000,25617,8918,0
1000001,11527,16616,0
1000002,25894,7015,0
1000003,15393,11126,0
1000004,26247,11173,0
...,...,...,...
1499995,16531,22024,0
1499996,17548,18092,1
1499997,4164,27359,0
1499998,8295,19919,0


In [15]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[1000000:1500000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_3', X1_onehot)
np.save('../../Data/X2_onehot_3', X2_onehot)
np.save('../../Data/Y_3', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:41, 1246.52it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# ######################STATUS ACTUAL################################

# Parte 4

In [16]:
training_ids_item2vec_df.iloc[1500000:2000000]

,Id_1,Id_2,Y
1500000,12900,26381,0
1500001,16830,25492,1
1500002,13913,26674,1
1500003,13257,19900,0
1500004,19234,17705,0
...,...,...,...
1999995,25866,19414,0
1999996,6937,5908,0
1999997,15532,5749,1
1999998,1151,24075,1


In [17]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[1500000:2000000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_4', X1_onehot)
np.save('../../Data/X2_onehot_4', X2_onehot)
np.save('../../Data/Y_4', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:44, 1234.83it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 5

In [18]:
training_ids_item2vec_df.iloc[2000000:2500000]

,Id_1,Id_2,Y
2000000,9445,12947,0
2000001,3743,19854,1
2000002,10907,5944,0
2000003,23436,22489,0
2000004,2897,11057,0
...,...,...,...
2499995,26470,12732,0
2499996,19545,23391,0
2499997,20146,26518,0
2499998,13931,4364,0


In [19]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[2000000:2500000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_5', X1_onehot)
np.save('../../Data/X2_onehot_5', X2_onehot)
np.save('../../Data/Y_5', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:39, 1251.49it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 6

In [20]:
training_ids_item2vec_df.iloc[2500000:3000000]

,Id_1,Id_2,Y
2500000,10993,14825,1
2500001,10576,17977,0
2500002,27971,624,0
2500003,14332,27622,0
2500004,13545,20586,0
...,...,...,...
2999995,8748,9477,1
2999996,14442,21194,1
2999997,19519,18870,0
2999998,26073,2927,1


In [22]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[2500000:3000000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_6', X1_onehot)
np.save('../../Data/X2_onehot_6', X2_onehot)
np.save('../../Data/Y_6', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:44, 1236.44it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 7

In [23]:
training_ids_item2vec_df.iloc[3000000:3500000]

,Id_1,Id_2,Y
3000000,100,15424,0
3000001,14496,4846,0
3000002,6636,19868,1
3000003,208,27613,1
3000004,21237,13575,0
...,...,...,...
3499995,7200,4547,0
3499996,3372,8090,0
3499997,10656,19493,0
3499998,25956,5581,0


In [24]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[3000000:3500000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_7', X1_onehot)
np.save('../../Data/X2_onehot_7', X2_onehot)
np.save('../../Data/Y_7', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:37, 1257.14it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 8

In [25]:
training_ids_item2vec_df.iloc[3500000:4000000]

,Id_1,Id_2,Y
3500000,16908,8975,0
3500001,23731,16984,0
3500002,4780,23255,0
3500003,4937,22390,0
3500004,7045,7251,0
...,...,...,...
3999995,27689,11060,1
3999996,23254,22367,0
3999997,24625,26829,1
3999998,7219,12094,1


In [26]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[3500000:4000000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_8', X1_onehot)
np.save('../../Data/X2_onehot_8', X2_onehot)
np.save('../../Data/Y_8', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:43, 1239.35it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 9

In [27]:
training_ids_item2vec_df.iloc[4000000:4500000]

,Id_1,Id_2,Y
4000000,2803,13988,1
4000001,14966,24918,0
4000002,15783,15880,0
4000003,2777,11023,0
4000004,21161,21573,0
...,...,...,...
4499995,8418,12115,0
4499996,2705,7692,0
4499997,1657,4991,0
4499998,4407,1879,1


In [28]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[4000000:4500000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_9', X1_onehot)
np.save('../../Data/X2_onehot_9', X2_onehot)
np.save('../../Data/Y_9', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:31, 1277.39it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 10

In [29]:
training_ids_item2vec_df.iloc[4500000:5000000]

,Id_1,Id_2,Y
4500000,23263,13093,0
4500001,11127,11964,1
4500002,9942,3890,1
4500003,25478,11656,0
4500004,11532,16240,1
...,...,...,...
4999995,8502,27613,1
4999996,22318,17113,0
4999997,9167,7334,0
4999998,5016,20009,0


In [30]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[4000000:4500000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_10', X1_onehot)
np.save('../../Data/X2_onehot_10', X2_onehot)
np.save('../../Data/Y_10', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:39, 1251.01it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 11

In [31]:
training_ids_item2vec_df.iloc[5000000:5500000]

,Id_1,Id_2,Y
5000000,9827,4719,1
5000001,8105,7618,0
5000002,24058,8717,0
5000003,21684,17931,0
5000004,11521,17716,0
...,...,...,...
5499995,23091,9232,0
5499996,8273,9139,0
5499997,18626,9910,0
5499998,27574,28097,0


In [32]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[5000000:5500000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_11', X1_onehot)
np.save('../../Data/X2_onehot_11', X2_onehot)
np.save('../../Data/Y_11', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:40, 1248.82it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 12

In [35]:
training_ids_item2vec_df.iloc[5500000:6000000]

,Id_1,Id_2,Y
5500000,26400,25789,1
5500001,24985,27051,1
5500002,20421,20246,0
5500003,9440,19150,1
5500004,26337,12980,1
...,...,...,...
5999995,3186,21423,1
5999996,8380,23433,0
5999997,4852,23377,0
5999998,25599,712,0


In [36]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[5500000:6000000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_12', X1_onehot)
np.save('../../Data/X2_onehot_12', X2_onehot)
np.save('../../Data/Y_12', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:48, 1225.09it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 13

In [37]:
training_ids_item2vec_df.iloc[6000000:6500000]

,Id_1,Id_2,Y
6000000,273,7943,0
6000001,19475,5386,1
6000002,15400,11228,1
6000003,1115,16903,0
6000004,9307,27173,0
...,...,...,...
6499995,16877,27065,1
6499996,11167,9981,0
6499997,18067,7776,0
6499998,7046,26143,0


In [38]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[6000000:6500000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_13', X1_onehot)
np.save('../../Data/X2_onehot_13', X2_onehot)
np.save('../../Data/Y_13', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:50, 1218.20it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 14

In [41]:
training_ids_item2vec_df.iloc[6500000:7000000]

,Id_1,Id_2,Y
6500000,14825,13739,1
6500001,21809,17834,0
6500002,21117,11905,1
6500003,8991,17008,1
6500004,2601,11618,1
...,...,...,...
6999995,8806,6899,0
6999996,27688,14429,0
6999997,18209,9199,1
6999998,8021,10971,0


In [42]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[6500000:7000000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_14', X1_onehot)
np.save('../../Data/X2_onehot_14', X2_onehot)
np.save('../../Data/Y_14', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:46, 1228.77it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 15

In [43]:
training_ids_item2vec_df.iloc[7000000:7500000]

,Id_1,Id_2,Y
7000000,12408,7780,0
7000001,23570,16758,0
7000002,1439,16385,1
7000003,24429,26419,1
7000004,385,15808,0
...,...,...,...
7499995,27256,27771,0
7499996,7308,15995,0
7499997,22424,27077,0
7499998,15318,28079,0


In [44]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[7000000:7500000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_15', X1_onehot)
np.save('../../Data/X2_onehot_15', X2_onehot)
np.save('../../Data/Y_15', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:43, 1240.04it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 16

In [45]:
training_ids_item2vec_df.iloc[7500000:8000000]

,Id_1,Id_2,Y
7500000,9720,13740,0
7500001,16818,3774,1
7500002,14791,20095,0
7500003,28030,22056,1
7500004,4719,9934,0
...,...,...,...
7999995,9692,18289,1
7999996,22867,4511,1
7999997,8540,12081,0
7999998,22812,23226,1


In [46]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[7500000:8000000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_16', X1_onehot)
np.save('../../Data/X2_onehot_16', X2_onehot)
np.save('../../Data/Y_16', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:38, 1255.92it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 17

In [47]:
training_ids_item2vec_df.iloc[8000000:8500000]

,Id_1,Id_2,Y
8000000,9829,14,0
8000001,18638,12030,0
8000002,6184,1926,0
8000003,13857,6251,1
8000004,22807,21233,0
...,...,...,...
8499995,15540,16173,0
8499996,23275,24454,1
8499997,2188,12548,1
8499998,24906,980,0


In [48]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[8000000:8500000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_17', X1_onehot)
np.save('../../Data/X2_onehot_17', X2_onehot)
np.save('../../Data/Y_17', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:38, 1254.50it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 18

In [49]:
training_ids_item2vec_df.iloc[8500000:9000000]

,Id_1,Id_2,Y
8500000,27538,8237,0
8500001,2100,22712,0
8500002,396,13679,1
8500003,17467,17931,0
8500004,1640,1597,1
...,...,...,...
8999995,26862,22415,0
8999996,19899,18031,0
8999997,162,17439,1
8999998,24774,2582,1


In [50]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[8500000:9000000], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_18', X1_onehot)
np.save('../../Data/X2_onehot_18', X2_onehot)
np.save('../../Data/Y_18', Y)

del X1_onehot
del X2_onehot
del Y

500000it [06:44, 1237.16it/s]


(500000, 904)
(500000, 904)
(500000, 1)


# Parte 19

In [51]:
training_ids_item2vec_df.iloc[9000000:]

,Id_1,Id_2,Y
9000000,10904,717,0
9000001,18684,4859,0
9000002,17312,10514,0
9000003,12953,19495,1
9000004,7194,7152,1
...,...,...,...
9027251,5465,10434,1
9027252,16097,23946,1
9027253,5199,3037,1
9027254,12604,10772,1


In [52]:
X1_onehot, X2_onehot, Y = create_vectos_training_embedding(training_ids_item2vec_df.iloc[9000000:], items_onehot_encoded_df)

print(X1_onehot.shape)
print(X2_onehot.shape)
print(Y.shape)

np.save('../../Data/X1_onehot_19', X1_onehot)
np.save('../../Data/X2_onehot_19', X2_onehot)
np.save('../../Data/Y_19', Y)

del X1_onehot
del X2_onehot
del Y

27256it [00:21, 1247.31it/s]


(27256, 904)
(27256, 904)
(27256, 1)
